In [1]:
from scipy import stats
import statsmodels.api as sm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['axes.unicode_minus']=False # 解决负号显示异常的问题
import arch

# from jqdata import macro

In [2]:
TWII            = pd.read_excel('^TWII2008.xlsx')
#TWII_price      = TWII['收盤價']
TWII_rrplot     = TWII['日報酬'][1:]*100
#TWII_rr     = np.log(np.array(TWII_price[1:len(TWII_price)]))  -  np.log(np.array(TWII_price[0:len(TWII_price)-1]))

# Data Preprocessing???  不會資料分析  F

In [3]:
## 分割為2000, 1的資料

train = []
test  = []
for data in range(0,len(TWII_rrplot)-1500):
################Training Data#################
    window_data = TWII_rrplot[data:1500+data]
                 
    train = train + [pd.DataFrame(window_data)]#加方括號區隔10000筆的資料
################Testing Data##################   
    test_data = (TWII_rrplot[data+1500])
    
    test = test + [test_data]

# GARCH(1,1)

In [12]:
def garchfcn(TWII_rrplot):
#     train_GARCH = TWII_rrplot[:-20]
#     test_GARCH  = TWII_rrplot[-20:]
    GARCH       = arch.arch_model(TWII_rrplot, vol='GARCH',p=1,q=1) 
    res_GARCH   = GARCH.fit()
    
#     res_GARCH.summary(), res_GARCH.params
    
    at        =  TWII_rrplot  - np.array(TWII_rrplot).mean()
    at_square =  at**2 
    
#    ini = res_GARCH.conditional_volatility[-1:] #上一個條件方差
  
#     for i in range(50):
#         new = res_GARCH.params['omega'] + res_GARCH.params['alpha[1]']*at_square['日報酬'][-(50+i):] + res_GARCH.params['beta[1]']*ini[-1:]
#         ini = np.append(ini,new)
#     vol_pre = np.percentile(ini[-50:],5)
    vol_pre  = res_GARCH.params['omega'] + res_GARCH.params['alpha[1]']*at_square['日報酬'][-1:] + res_GARCH.params['beta[1]']*res_GARCH.conditional_volatility[-1:]
    
    VaR_pre  = res_GARCH.params[0]  + np.sqrt(vol_pre[window])*-1.645
    
    result   = VaR_pre
    return result

In [13]:
riskresult = {}
for window in range(1500,len(TWII_rrplot)):
    riskresult_train = []
    training = train[window-1500]
    varisk_intake = garchfcn(training)
    riskresult_train.append(varisk_intake)
    
    riskresult[window] = riskresult_train

Iteration:      1,   Func. Count:      6,   Neg. LLF: 2385.966240679307
Iteration:      2,   Func. Count:     15,   Neg. LLF: 2384.879213297816
Iteration:      3,   Func. Count:     25,   Neg. LLF: 2384.871538039605
Iteration:      4,   Func. Count:     32,   Neg. LLF: 2382.9380191801565
Iteration:      5,   Func. Count:     39,   Neg. LLF: 2380.7747364363313
Iteration:      6,   Func. Count:     46,   Neg. LLF: 2379.567646578321
Iteration:      7,   Func. Count:     53,   Neg. LLF: 2376.3779980015797
Iteration:      8,   Func. Count:     61,   Neg. LLF: 2376.3094277103482
Iteration:      9,   Func. Count:     68,   Neg. LLF: 2375.861652120338
Iteration:     10,   Func. Count:     75,   Neg. LLF: 2375.8475882886014
Iteration:     11,   Func. Count:     82,   Neg. LLF: 2375.800906563263
Iteration:     12,   Func. Count:     88,   Neg. LLF: 2375.800071928471
Iteration:     13,   Func. Count:     94,   Neg. LLF: 2375.800032209927
Iteration:     14,   Func. Count:    100,   Neg. LLF: 2375.

Iteration:      1,   Func. Count:      6,   Neg. LLF: 2384.44321789294
Iteration:      2,   Func. Count:     15,   Neg. LLF: 2383.538267908486
Iteration:      3,   Func. Count:     22,   Neg. LLF: 2381.8106237294114
Iteration:      4,   Func. Count:     29,   Neg. LLF: 2380.4281041404333
Iteration:      5,   Func. Count:     36,   Neg. LLF: 2379.8352251010538
Iteration:      6,   Func. Count:     43,   Neg. LLF: 2377.945986199946
Iteration:      7,   Func. Count:     50,   Neg. LLF: 2377.5151637997487
Iteration:      8,   Func. Count:     57,   Neg. LLF: 2377.2813136982904
Iteration:      9,   Func. Count:     64,   Neg. LLF: 2377.2457123053828
Iteration:     10,   Func. Count:     71,   Neg. LLF: 2377.116821956647
Iteration:     11,   Func. Count:     77,   Neg. LLF: 2377.0026482746152
Iteration:     12,   Func. Count:     83,   Neg. LLF: 2376.9792575472793
Iteration:     13,   Func. Count:     89,   Neg. LLF: 2376.978430324065
Iteration:     14,   Func. Count:     95,   Neg. LLF: 237

Iteration:      1,   Func. Count:      6,   Neg. LLF: 2371.415129276119
Iteration:      2,   Func. Count:     15,   Neg. LLF: 2370.6133737028836
Iteration:      3,   Func. Count:     25,   Neg. LLF: 2370.608771903954
Iteration:      4,   Func. Count:     32,   Neg. LLF: 2368.8406729688263
Iteration:      5,   Func. Count:     39,   Neg. LLF: 2367.6177295121943
Iteration:      6,   Func. Count:     46,   Neg. LLF: 2366.447582509131
Iteration:      7,   Func. Count:     53,   Neg. LLF: 2364.7257786237687
Iteration:      8,   Func. Count:     60,   Neg. LLF: 2364.268523148002
Iteration:      9,   Func. Count:     67,   Neg. LLF: 2363.8573925590704
Iteration:     10,   Func. Count:     75,   Neg. LLF: 2363.8200547241877
Iteration:     11,   Func. Count:     81,   Neg. LLF: 2363.7779971997743
Iteration:     12,   Func. Count:     87,   Neg. LLF: 2363.76310217609
Iteration:     13,   Func. Count:     93,   Neg. LLF: 2363.762985606089
Iteration:     14,   Func. Count:     99,   Neg. LLF: 2363

Iteration:     12,   Func. Count:     87,   Neg. LLF: 2354.1973492228863
Iteration:     13,   Func. Count:     93,   Neg. LLF: 2354.197129162443
Iteration:     14,   Func. Count:     99,   Neg. LLF: 2354.1971245761943
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 2354.1971238051146
            Iterations: 14
            Function evaluations: 100
            Gradient evaluations: 14
Iteration:      1,   Func. Count:      6,   Neg. LLF: 2360.7661828456266
Iteration:      2,   Func. Count:     15,   Neg. LLF: 2359.967467657056
Iteration:      3,   Func. Count:     22,   Neg. LLF: 2358.23060765181
Iteration:      4,   Func. Count:     29,   Neg. LLF: 2357.0000086665605
Iteration:      5,   Func. Count:     36,   Neg. LLF: 2356.375294349318
Iteration:      6,   Func. Count:     43,   Neg. LLF: 2353.856638788655
Iteration:      7,   Func. Count:     51,   Neg. LLF: 2353.777126127565
Iteration:      8,   Func. Count:     58,   Neg. LLF: 2353.373784

Iteration:      1,   Func. Count:      6,   Neg. LLF: 2350.845163494419
Iteration:      2,   Func. Count:     15,   Neg. LLF: 2349.932501599269
Iteration:      3,   Func. Count:     22,   Neg. LLF: 2348.2367933015967
Iteration:      4,   Func. Count:     29,   Neg. LLF: 2347.311888016921
Iteration:      5,   Func. Count:     36,   Neg. LLF: 2346.607591665041
Iteration:      6,   Func. Count:     46,   Neg. LLF: 2346.56651243916
Iteration:      7,   Func. Count:     53,   Neg. LLF: 2343.866127114692
Iteration:      8,   Func. Count:     60,   Neg. LLF: 2343.60774119069
Iteration:      9,   Func. Count:     67,   Neg. LLF: 2342.902860564467
Iteration:     10,   Func. Count:     74,   Neg. LLF: 2342.8006318582848
Iteration:     11,   Func. Count:     80,   Neg. LLF: 2342.762646502517
Iteration:     12,   Func. Count:     86,   Neg. LLF: 2342.76108197317
Iteration:     13,   Func. Count:     92,   Neg. LLF: 2342.761067916108
Optimization terminated successfully.    (Exit mode 0)
          

...

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1723.7572080910709
Iteration:      2,   Func. Count:     16,   Neg. LLF: 1723.4914330151269
Iteration:      3,   Func. Count:     26,   Neg. LLF: 1723.3981768728293
Iteration:      4,   Func. Count:     35,   Neg. LLF: 1722.8851567682093
Iteration:      5,   Func. Count:     43,   Neg. LLF: 1722.48267878976
Iteration:      6,   Func. Count:     49,   Neg. LLF: 1722.4300776908608
Iteration:      7,   Func. Count:     55,   Neg. LLF: 1722.4264828671744
Iteration:      8,   Func. Count:     61,   Neg. LLF: 1722.4264258219334
Iteration:      9,   Func. Count:     67,   Neg. LLF: 1722.4264118627966
Iteration:     10,   Func. Count:     73,   Neg. LLF: 1722.426410044993
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 1722.4264100452867
            Iterations: 10
            Function evaluations: 73
            Gradient evaluations: 10
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1723.338

Iteration:     10,   Func. Count:     73,   Neg. LLF: 1720.032848262641
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 1720.032848262994
            Iterations: 10
            Function evaluations: 73
            Gradient evaluations: 10
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1725.8250018590375
Iteration:      2,   Func. Count:     16,   Neg. LLF: 1725.5207470983985
Iteration:      3,   Func. Count:     26,   Neg. LLF: 1725.4517504913324
Iteration:      4,   Func. Count:     35,   Neg. LLF: 1724.8666536354206
Iteration:      5,   Func. Count:     43,   Neg. LLF: 1724.5002481064264
Iteration:      6,   Func. Count:     49,   Neg. LLF: 1724.4445347133337
Iteration:      7,   Func. Count:     55,   Neg. LLF: 1724.440663172435
Iteration:      8,   Func. Count:     61,   Neg. LLF: 1724.4406126966733
Iteration:      9,   Func. Count:     67,   Neg. LLF: 1724.4406053245111
Iteration:     10,   Func. Count:     73,   Neg. LLF: 1724.440

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1723.086238875247
Iteration:      2,   Func. Count:     16,   Neg. LLF: 1722.805522669808
Iteration:      3,   Func. Count:     26,   Neg. LLF: 1722.7210022569861
Iteration:      4,   Func. Count:     35,   Neg. LLF: 1722.0854578922317
Iteration:      5,   Func. Count:     43,   Neg. LLF: 1721.735591837968
Iteration:      6,   Func. Count:     49,   Neg. LLF: 1721.6682441002758
Iteration:      7,   Func. Count:     55,   Neg. LLF: 1721.6599957936126
Iteration:      8,   Func. Count:     61,   Neg. LLF: 1721.6598958586458
Iteration:      9,   Func. Count:     67,   Neg. LLF: 1721.6598905423507
Iteration:     10,   Func. Count:     73,   Neg. LLF: 1721.6598890246778
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 1721.6598890254809
            Iterations: 10
            Function evaluations: 73
            Gradient evaluations: 10
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1723.025

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 1718.4433113941454
            Iterations: 11
            Function evaluations: 79
            Gradient evaluations: 11
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1718.2125678489601
Iteration:      2,   Func. Count:     16,   Neg. LLF: 1717.9130796493873
Iteration:      3,   Func. Count:     26,   Neg. LLF: 1717.8443357454742
Iteration:      4,   Func. Count:     35,   Neg. LLF: 1717.1952516244787
Iteration:      5,   Func. Count:     43,   Neg. LLF: 1716.8586915947644
Iteration:      6,   Func. Count:     49,   Neg. LLF: 1716.7929596153208
Iteration:      7,   Func. Count:     55,   Neg. LLF: 1716.7856339508273
Iteration:      8,   Func. Count:     61,   Neg. LLF: 1716.7855532219603
Iteration:      9,   Func. Count:     67,   Neg. LLF: 1716.7855522067862
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 1716.7855522044133
            Iterations

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1715.6962223066243
Iteration:      2,   Func. Count:     16,   Neg. LLF: 1715.3481838623125
Iteration:      3,   Func. Count:     26,   Neg. LLF: 1715.289523891423
Iteration:      4,   Func. Count:     35,   Neg. LLF: 1714.5849034961936
Iteration:      5,   Func. Count:     43,   Neg. LLF: 1714.2157344467137
Iteration:      6,   Func. Count:     49,   Neg. LLF: 1714.151241791936
Iteration:      7,   Func. Count:     55,   Neg. LLF: 1714.1463858477268
Iteration:      8,   Func. Count:     61,   Neg. LLF: 1714.145824948479
Iteration:      9,   Func. Count:     67,   Neg. LLF: 1714.1456877596027
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 1714.1456870641791
            Iterations: 9
            Function evaluations: 68
            Gradient evaluations: 9
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1715.6454111278395
Iteration:      2,   Func. Count:     16,   Neg. LLF: 1715.30429

In [14]:
riskresult

{1500: [-1.1571224528077932],
 1501: [-1.3244689453263159],
 1502: [-1.6021474372223004],
 1503: [-1.4896655998191457],
 1504: [-1.5052234414630645],
 1505: [-1.4643438044429702],
 1506: [-1.4556020498578783],
 1507: [-1.47921848982567],
 1508: [-1.4534510259631597],
 1509: [-1.4433355269015136],
 1510: [-1.4158562382949706],
 1511: [-1.4072461695901834],
 1512: [-1.3879737221185287],
 1513: [-1.3923104066745775],
 1514: [-1.4069355825105918],
 1515: [-1.3858076032972744],
 1516: [-1.3660739536628914],
 1517: [-1.3526703776562363],
 1518: [-1.3429266856924909],
 1519: [-1.335348236368789],
 1520: [-1.3364525563712069],
 1521: [-1.3615585067746518],
 1522: [-1.3761002232688513],
 1523: [-1.3414818562291067],
 1524: [-1.345635902602291],
 1525: [-1.3338030950195898],
 1526: [-1.3169754558704923],
 1527: [-1.3299129617041643],
 1528: [-1.3342198362996813],
 1529: [-1.3117874943504033],
 1530: [-1.3028704360491339],
 1531: [-1.3006405253786686],
 1532: [-1.3527797544946467],
 1533: [-1.324

In [15]:
riskresult_data = []
for index in range(1500,1500+len(riskresult)):
        riskresult_data = riskresult_data + riskresult[index]

print(np.array(riskresult_data).mean())
print(np.array(riskresult_data).std())
print(np.array(riskresult_data).min())
print(np.array(riskresult_data).max())

-1.4094296055160818
0.18307278386203626
-2.4662807387716406
-1.126199595036787


In [16]:
len(riskresult_data)

1141

In [17]:
##試算 常態前000筆資料 
failure      = 0
failure_day  = []
failure_rr   = []
AQLF         = 0
for window in range(1500,1500+len(riskresult_data)):
    if riskresult[window] > test[window-1500]:
        failure    = failure +1
        failure_day = failure_day  + [window]
        failure_rr  = failure_rr   + [test[window-1500]]
        AQLF     = AQLF    +   1+(riskresult_data[window-1500] - test[window-1500])**2
print(failure)
print(failure/1141)
# print(failure_day)
#print(np.array(failure_rr).mean())
print(AQLF/1141)

41
0.03593339176161262
0.05681164065547377


In [18]:
real_fail_rate = failure/1141

LR =-2*np.log10((0.05**failure)*(0.95**(1141-failure)))+2*np.log10((real_fail_rate**failure)*(1-real_fail_rate)**(1141-failure))
LR

2.2788710457290335

In [12]:
file=open('garchpredict','w')  
file.write(str(riskresult_data));  
file.close() 